# Trabajo Práctico 3: Detectando barcos en el agua

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from pathlib import Path
import pickle
import timeit

%matplotlib inline

In [ ]:
# configuraciones generales de las imagenes
PICTURE_SIZE = 80
CHANNELS = 'rgb' # canales
PATH_IMAGE = 'imagenes_tp3_cv/' #ubicación de las imágenes
PATH_PKL = 'Archivos_pkl'

INPUT_COLUMNS = []
# Crea lista el nombre de las columas
for color in CHANNELS:
    INPUT_COLUMNS.extend(['%s%i' % (color, i) 
                              for i in range(PICTURE_SIZE ** 2)]) # For 80 * 80

_Columns = INPUT_COLUMNS
#INPUT_COLUMNS.append('Barco')
#INPUT_COLUMNS.append('ImageName')

print(INPUT_COLUMNS)

Cada fila es una imagen.
La imagen tiene 80 x 80 pixeles, y cada pixel tiene 3 valores, entonces tenemos 80 x 80 x 3 = 19.200‬ columnas.

In [ ]:
print('Imagen')
imagen = Image.open(PATH_IMAGE + '0__20160622_170157_0c64__-122.485753590087_37.835957669247584.png')
imagen

In [ ]:
cant_image = len(os.listdir(PATH_IMAGE)) 
print('Hay ' + str(cant_image) + ' imagenes en la carpeta ' + PATH_IMAGE)

In [ ]:
#comienza a correr el tiempo
start_time = timeit.default_timer()

#lista de listas
list_of_list = []
# recorro todas las imagenes de la carpeta
for archivo in os.listdir(PATH_IMAGE):
    #print(archivo)
    L_rojo = []
    L_verde = []
    L_azul = []
    
    Name_Image = archivo #nombre del archivo
    Barco = archivo[0] #nombre del archivo
    
    imagen = Image.open(PATH_IMAGE + archivo) # abro la imagen
    pixeles = list(imagen.getdata()) # trasformo a pixeles rgb
    
    for pixel in pixeles:
        rojo, verde, azul = pixel
        L_rojo.append(rojo)
        L_verde.append(verde)
        L_azul.append(azul)

    
    L_pixeles = []
    
    L_pixeles.extend(L_rojo)
    L_pixeles.extend(L_verde)
    L_pixeles.extend(L_azul)
    L_pixeles.append(Barco)
    L_pixeles.append(Name_Image)
    
    list_of_list.append(L_pixeles)

#Tranforma la lista a dataframe
df = pd.DataFrame(list_of_list)

elapsed = timeit.default_timer() - start_time

print('Tiempo de ejecución ' + str(elapsed))
df.head(5)

In [ ]:
# Cambia el nombre de las columnas
df.columns = INPUT_COLUMNS
df.head(5)

In [ ]:
# genero el archivo
df.to_pickle('Archivos_pkl/data_{}x{}.pkl'.format(PICTURE_SIZE, PICTURE_SIZE)) 

In [ ]:
data_barco = pd.read_pickle('Archivos_pkl/data_{}x{}.pkl'.format(PICTURE_SIZE, PICTURE_SIZE))
data_barco.shape

In [ ]:
Etiquetas_barco = list(data_barco.Barco.unique())
Etiquetas_barco

In [ ]:
def barcos_imagenes(samples, title='file'):
    for index, sample in samples.iterrows():
        if title is not None:
            if isinstance(title, str):
                title = [title, ]
            title_text = ', '.join(str(sample[title_field]) for title_field in title)
            plt.title(title_text)

        sample_as_grid = sample[INPUT_COLUMNS].values.reshape(len(CHANNELS), PICTURE_SIZE, PICTURE_SIZE).astype(np.float)
        sample_as_grid = np.transpose(sample_as_grid, (1, 2, 0)) / 255
        
        plt.axis('off')
        plt.imshow(sample_as_grid, interpolation='nearest')

        plt.show()

In [ ]:
barcos_imagenes(data_barco.sample(5), 'Barco')

In [ ]:
train, not_train = train_test_split(data_barco, test_size=0.4)
validation, test = train_test_split(not_train, test_size=0.5)

sets = (
    ('train', train),
    ('test', test),
)

print (' Train:', train.shape, '\n', 'Validation:', validation.shape, '\n', 'Test:', test.shape)

In [ ]:
#Como quedan balanceado los datos
f,ax=plt.subplots(1,3,figsize=(16,6))

ax[0].set_title('Train')
train.Barco.value_counts().plot.pie(autopct='%1.0f%%', figsize=(8,8), ax=ax[0])

ax[1].set_title('Validation')
validation.Barco.value_counts().plot.pie(autopct='%1.0f%%', figsize=(8,8), ax=ax[1])

ax[2].set_title('Test')
test.Barco.value_counts().plot.pie(autopct='%1.0f%%', figsize=(8,8), ax=ax[2])

In [ ]:
def extract_inputs(dataset):
    return dataset[_Columns].values / 255
   
def extract_outputs(dataset):
    is_barco_columns = [(dataset.Barco == barco).values for barco in Etiquetas_barco]
    return np.array(is_barco_columns).T

In [ ]:
model = Sequential([
    Dense(10, activation='tanh', input_shape=(len(_Columns), )),
    #Convolution2D(8, (4, 4), activation='relu', input_shape=(PICTURE_SIZE, PICTURE_SIZE, len(CHANNELS)),),
    #Convolution2D(8, (4, 4), activation='relu'),
    #MaxPooling2D(pool_size=(4, 4)),
    #Dropout(0.2),
    #Flatten(),
        
    Dense(10, activation='tanh'),
    Dropout(0.2),
    Dense(10, activation='tanh'),
    Dropout(0.2),

    Dense(len(Etiquetas_barco), activation='softmax'),
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy',],
)

In [ ]:
history = model.fit(
    extract_inputs(train), 
    extract_outputs(train), 
    epochs=5,
    batch_size=128,
    validation_data=(
        extract_inputs(test),
        extract_outputs(test),
    )
)

In [ ]:
def add_predictions(dataset):
    predictions = model.predict(extract_inputs(dataset))

    for numero_barco, barco in enumerate(Etiquetas_barco):
        dataset[barco] = predictions[:, numero_barco]
    
    dataset['prediction'] = dataset[Etiquetas_barco].idxmax(axis=1)
    dataset['prediction_confidence'] = dataset[Etiquetas_barco].max(axis=1)
    dataset['correct'] = dataset.prediction == dataset.Barco

In [ ]:
for set_name, set_data in sets:
    add_predictions(set_data)
    
    print('#' * 25, set_name, '#' * 25)
    print('accuracy', accuracy_score(set_data.Barco, set_data.prediction))
       
    plt.figure(figsize=(3,4))
        
    plt.xticks([0, 1, 2, 3], Etiquetas_barco, rotation=45)
    plt.yticks([0, 1, 2, 3], Etiquetas_barco)
    plt.xlabel('Predicted class')
    plt.ylabel('True class')

    plt.title(set_name)

    plt.imshow(
        confusion_matrix(set_data.Barco, set_data.prediction), 
        cmap=plt.cm.Blues,
        interpolation='nearest',
    )

    plt.show()

In [ ]:
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='test')
plt.title('Accuracy over train epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

In [ ]:
print("Imagenes bien clasificadas")
barcos_imagenes(test[test.Barco == test.prediction].sample(10), title=['prediction', 'prediction_confidence'])

In [ ]:
print("Imagenes mal clasificadas")
barcos_imagenes(test[test.Barco != test.prediction].sample(10), title=['prediction', 'prediction_confidence'])